## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-14-02-19-31 +0000,nyt,N.I.H. Worker Who Criticized Trump Health Poli...,https://www.nytimes.com/2025/11/13/us/politics...
1,2025-11-14-02-18-00 +0000,wsj,China’s Economic Growth Momentum Slowed in Oct...,https://www.wsj.com/economy/chinas-economic-gr...
2,2025-11-14-02-16-00 +0000,nypost,Ammonia leak from tanker truck sends dozens t...,https://nypost.com/2025/11/13/us-news/ammonia-...
3,2025-11-14-02-11-09 +0000,nyt,Justice Dept. Memo Blessing Boat Strikes Is Sa...,https://www.nytimes.com/2025/11/13/us/politics...
4,2025-11-14-02-00-00 +0000,wsj,Inside the Frantic Push to Reverse Verizon’s D...,https://www.wsj.com/business/telecom/inside-th...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,37
240,shutdown,18
45,new,18
114,epstein,17
44,california,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
319,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...,113
206,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...,108
328,2025-11-13-04-11-30 +0000,wapo,Longest government shutdown in U.S. history en...,https://www.washingtonpost.com/business/2025/1...,88
234,2025-11-13-13-58-49 +0000,nypost,Trump-obsessed writer Michael Wolff encouraged...,https://nypost.com/2025/11/13/us-news/michael-...,84
185,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...,83


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
319,113,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...
206,61,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...
254,50,2025-11-13-12-13-02 +0000,nypost,Jeffrey Epstein gave ex-Clinton Treasury Secre...,https://nypost.com/2025/11/13/us-news/jeffrey-...
43,49,2025-11-13-23-34-00 +0000,wsj,Boeing Defense Workers Approve New Five-Year C...,https://www.wsj.com/business/boeing-defense-wo...
185,46,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...
196,44,2025-11-13-16-58-27 +0000,nypost,Disney shares plunge 9% as ABC’s ‘World News T...,https://nypost.com/2025/11/13/business/disney-...
285,43,2025-11-13-10-30-00 +0000,wsj,Heard on the Street: Borrowing from a builder ...,https://www.wsj.com/economy/housing/builders-c...
121,37,2025-11-13-20-54-33 +0000,latimes,Starbucks union launches Red Cup Day strike at...,https://www.latimes.com/business/story/2025-11...
73,30,2025-11-13-22-32-00 +0000,wsj,Applied Materials Forecasts Higher Sales in Se...,https://www.wsj.com/business/earnings/applied-...
183,30,2025-11-13-17-20-59 +0000,bbc,Starmer's chief of staff McSweeney not leaving...,https://www.bbc.com/news/articles/cn8vn4vv87xo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
